In [79]:
import nltk # Used for preprocessing
import string # Used for preprocessing
import numpy as np # Used for saving
import pandas as pd # Used for dataframe

from nltk.corpus import stopwords # Used for preprocessing
from nltk.stem   import WordNetLemmatizer # Used for preprocessing
from nltk.tokenize import word_tokenize # Used for preprocessing
from sklearn.model_selection import train_test_split # Used for splitting the data

from sklearn.svm import SVC # Used for SVM modeling
from sklearn.model_selection import cross_validate # Used for cross-validation
from sklearn.metrics import classification_report # Used for model evaluation

from nltk.tokenize import sent_tokenize, word_tokenize
import warnings
import gensim
from gensim.models import Word2Vec
import joblib # For saving the model

In [80]:
# Loading the dataset

df = pd.read_csv('WELFake_Dataset.csv', index_col = 0)
df = df.dropna()

In [82]:
# Splitting data

X = df['text']
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=21)

X_train = X_train.fillna('')
X_test = X_test.fillna('')

In [81]:
# Preprocessing function

def preprocess_text(text):
    text = ''.join([c for c in text if c not in string.punctuation and c not in string.digits])
    tokens = word_tokenize(text, 'english')
    lemmatiser = WordNetLemmatizer()
    lemmatized = [lemmatiser.lemmatize(word) for word in tokens]
    sw = stopwords.words('english')
    stopped = [word for word in lemmatized if word.lower() not in sw]
    return stopped

# GloVe

In [162]:
from annoy import AnnoyIndex
import pickle

Embedding_Dim = 50
Glove_File_Pr = '/Users/pietervanbrakel/Thesis/data/glove.6B.50d.{}'

def build_index():
    num_trees = 10
    idx = AnnoyIndex(Embedding_Dim)
    index_to_word = {}
    with open(Glove_File_Pr.format('txt')) as f:
        for i, line in enumerate(f):
            fields = line.rstrip().split(' ')
            vec = [float(x) for x in fields[1:]]
            idx.add_item(i, vec)
            index_to_word[i] = fields[0]
            
    idx.build(num_trees)
    idx.save(Glove_File_Pr.format('.idx'))
    pickle.dump(index_to_word, open(Glove_File_Pr.format('.i2w'), mode = 'wb'))
    
def search(query, top_n=10):
    idx = AnnoyIndex(Embedding_Dim)
    idx.load(Glove_File_Pr.format('.idx'))
    index_to_word = pickle.load(open(Glove_File_Pr.format('.i2w'),
                                    mode = 'rb'))
    word_to_index = {word: index for index, word in index_to_word.items()}
    query_id = word_to_index[query]
    word_ids = idx.get_nns_by_item(query_id, top_n)
    for word_id in word_ids:
        print(index_to_word[word_id])

In [163]:
build_index()
search('dog')

<ipython-input-162-5ae4c4b2291a>:9: FutureWarning: The default argument for metric will be removed in future version of Annoy. Please pass metric='angular' explicitly.
  idx = AnnoyIndex(Embedding_Dim)
<ipython-input-162-5ae4c4b2291a>:23: FutureWarning: The default argument for metric will be removed in future version of Annoy. Please pass metric='angular' explicitly.
  idx = AnnoyIndex(Embedding_Dim)


dog
cat
dogs
puppy
pet
rabbit
pig
snake
baby
boy


In [164]:
file = '/Users/pietervanbrakel/Thesis/data/glove.6B.50d.txt'
with open(file) as f:
    content = f.readlines()
content = [x.split(" ") for x in content]

In [165]:
search('profit')

<ipython-input-162-5ae4c4b2291a>:23: FutureWarning: The default argument for metric will be removed in future version of Annoy. Please pass metric='angular' explicitly.
  idx = AnnoyIndex(Embedding_Dim)


profit
earnings
profits
shares
sales
revenue
net
share
gains
revenues


In [166]:
counter = -1
line_list = []

for line in X_train[:5]:
    word_list = []
    counter += 1
    print( (counter/len(X_train[:5])) * 100, "%")
    for word1 in preprocess_text(line):
        for word2 in content:
            if word2[0] == word1:
                word_list.append(word2[1:])
    line_list.append(word_list)

0.0 %
20.0 %
40.0 %
60.0 %
80.0 %


In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

# Building the dictionary
tokenizer = Tokenizer()
for line in X_train:
    tokenizer.fit_on_texts(preprocess_text(line))

In [ ]:
print(len(tokenizer))

In [172]:
def embedding_for_vocab(filepath, word_index,
                        embedding_dim):
    vocab_size = len(word_index) + 1
      
    # Adding again 1 because of reserved 0 index
    embedding_matrix_vocab = np.zeros((vocab_size,
                                       embedding_dim))
  
    with open(filepath, encoding="utf8") as f:
        for line in f:
            word, *vector = line.split()
            if word in word_index:
                idx = word_index[word]
                embedding_matrix_vocab[idx] = np.array(
                    vector, dtype=np.float32)[:embedding_dim]
  
    return embedding_matrix_vocab

In [173]:
embedding_dim = 50
embedding_matrix_vocab = embedding_for_vocab(
    '/Users/pietervanbrakel/Thesis/data/glove.6B.50d.txt', tokenizer.word_index,
  embedding_dim)

In [180]:
print("Dense vector for first word is => ",
      len(embedding_matrix_vocab[2]))

Dense vector for first word is =>  50


In [177]:
svm = SVC()
scores = cross_validate(svm, embedding_matrix_vocab[:5], y_train[:5], scoring=['f1', 'accuracy'], cv=5, n_jobs=-1) # Scoring the training

ValueError: n_splits=5 cannot be greater than the number of members in each class.